# Build baseline tfrs model 

In [2]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 

In [1]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_io as tfio

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

## Create Small Dataset for testing

### features and helper functions

In [3]:
candidate_features = {
        'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
}

In [63]:
cont_feats = {
    'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'pos_seed_track': tf.io.FixedLenFeature(dtype=tf.int64, shape=()),
    'track_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'pid': tf.io.FixedLenFeature(dtype=tf.int64, shape=()),
    'name': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'collaborative': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    # 'duration_ms_seed_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'n_songs_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'num_artists_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'num_albums_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'description_pl': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
}
    ###ragged

seq_feats = {
    'track_name_pl': tf.io.RaggedFeature(tf.string),
    'artist_name_pl': tf.io.RaggedFeature(tf.string),
    'album_name_pl': tf.io.RaggedFeature(tf.string),
    'track_uri_pl': tf.io.RaggedFeature(tf.string),
    'duration_ms_songs_pl': tf.io.RaggedFeature(tf.float32),
    'artist_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artists_followers_pl': tf.io.RaggedFeature(tf.float32),
    'track_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artist_genres_pl': tf.io.RaggedFeature(tf.string),
}


### Playlist dataset

In [64]:
## testing output
from google.cloud import storage

client = storage.Client()
# # gs://spotify-beam-v3/v3/candidates/*.tfrecords

BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/valid/'

valid_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
raw_dataset = tf.data.TFRecordDataset(valid_files[:3])

def parse_tfrecord(example):
    example = tf.io.parse_single_sequence_example(
        example, 
        context_features=cont_feats,
        sequence_features=seq_feats
    )
    return example

In [78]:

parsed_dataset = raw_dataset.map(parse_tfrecord)


MAX_PLAYLIST_LENGTH = 375


# gives: 
# array([[ 1,  2, -1, -1],
#       [ 3,  4, -1, -1]], dtype=int32)

def pad_up_to(t, max_in_dims=[1 ,MAX_PLAYLIST_LENGTH], constant_value=''):
    s = tf.shape(t)
    paddings = [[0, m-s[i]] for (i,m) in enumerate(max_in_dims)]
    return tf.pad(t, paddings, 'CONSTANT', constant_values=constant_value)

def return_padded_tensors(context, data):
    
        a = pad_up_to(tf.reshape(data['track_name_pl'], shape=(1,-1)) , constant_value='')
        b = pad_up_to(tf.reshape(data['artist_name_pl'], shape=(1,-1)) , constant_value='')
        c = pad_up_to(tf.reshape(data['album_name_pl'], shape=(1,-1)) , constant_value='')
        d = pad_up_to(tf.reshape(data['track_uri_pl'], shape=(1, -1,)) , constant_value='')
        e = pad_up_to(tf.reshape(data['duration_ms_songs_pl'], shape=(1,-1)) , constant_value=-1.)
        f = pad_up_to(tf.reshape(data['artist_pop_pl'], shape=(1,-1)) , constant_value=-1.)
        g = pad_up_to(tf.reshape(data['artists_followers_pl'], shape=(1,-1)) , constant_value=-1.)
        h = pad_up_to(tf.reshape(data['track_pop_pl'], shape=(1,-1)) , constant_value=-1.)
        i = pad_up_to(tf.reshape(data['artist_genres_pl'], shape=(1,-1)) , constant_value='')
        
        padded_data = context.copy()
        padded_data['track_name_pl'] = a
        padded_data['artist_name_pl'] = b
        padded_data['album_name_pl'] = c
        padded_data['track_uri_pl'] = d
        padded_data['duration_ms_songs_pl'] = e
        padded_data['artist_pop_pl'] = f
        padded_data['artists_followers_pl'] = g
        padded_data['track_pop_pl'] = h
        padded_data['artist_genres_pl'] = i
        
        return padded_data
parsed_dataset_padded = parsed_dataset.map(return_padded_tensors)   

In [79]:
for features in parsed_dataset_padded.skip(3).take(1):
    pprint(features)
    print("_______________")

{'album_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'Word Of Mouth'>,
 'album_name_pl': <tf.Tensor: shape=(1, 375), dtype=string, numpy=
array([[b'Stranglehold', b'Roadhouse 01', b'A Song For Every Moon',
        b'Uncomfortable - Single', b'A Song For Every Moon', b'Freudian',
        b'Michl', b'Salt', b'blkswn', b'blkswn', b'blkswn', b'blkswn',
        b'Blonde', b'Chanel', b'Kiddo', b'I Learnt Some Jazz Today',
        b"Say It Here, While It's Safe",
        b'Piece of Cake (feat. Harrison Sands and Shota)',
        b'Land of Lights', b'Blonde', b'Nat Love', b'Pine & Ginger',
        b'She Say', b'S!ck S!ck S!ck', b'Telefone', b'Nocturnal',
        b'Hotel Allan', b'alarm (prod. no sentences)',
        b'Buttermilk - EP', b'Restoration of an American Idol',
        b'Into the Flame', b'The Thirst (feat. Shiz)', b'Urban Flora',
        b'A Song For Every Moon', b"Sex n' Drugs", b'Thoughts of You',
        b"Searchin'", b'Tell Me Why', b'Kiddo', b'Por Vida', b'grey',
     

### Candidate Track dataset

In [71]:
BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/candidates/'

candidate_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
candidate_dataset = tf.data.TFRecordDataset(candidate_files)

def parse_candidate_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=candidate_features
    )
    return example

# parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn, num_parallel_calls=-1)

parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn) ### THIS NEEDS TO BE FIXED SO THE UNIQUE PRODUCT DATASET HAS THE SAME FIELD NAMES (goes thru the same model)

In [38]:
for features in parsed_candidate_dataset.take(1):
    pprint(features)
    print("_______________")

{'album_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'The Sound of Everything Rmx'>,
 'album_uri_can': <tf.Tensor: shape=(), dtype=string, numpy=b'spotify:album:4a8tMD6qq6GUuUwNae38VI'>,
 'artist_followers_can': <tf.Tensor: shape=(), dtype=float32, numpy=277649.0>,
 'artist_genres_can': <tf.Tensor: shape=(), dtype=string, numpy=b"'downtempo', 'electronica', 'funk', 'latin alternative', 'nu jazz', 'nu-cumbia', 'trip hop', 'world'">,
 'artist_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'Quantic'>,
 'artist_pop_can': <tf.Tensor: shape=(), dtype=float32, numpy=64.0>,
 'artist_uri_can': <tf.Tensor: shape=(), dtype=string, numpy=b'spotify:artist:5ZMwoAjeDtLJ0XRwRTgaK8'>,
 'duration_ms_can': <tf.Tensor: shape=(), dtype=float32, numpy=267130.0>,
 'track_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'The Sound of Everything - Watch TV & Se\xc3\xb1orlobo Remix'>,
 'track_pop_can': <tf.Tensor: shape=(), dtype=float32, numpy=53.0>,
 'track_uri_can': <tf.Tensor: shape=(),

### Vocab files

In [39]:
BUCKET_NAME = 'spotify-v1'
FILE_PATH = 'vocabs/v1_string_vocabs'
FILE_NAME = 'string_vocabs_v1_20220705-202905.txt'
DESTINATION_FILE = 'downloaded_vocabs.txt'

client = storage.Client()

with open(f'{DESTINATION_FILE}', 'wb') as file_obj:
    client.download_blob_to_file(
        f'gs://{BUCKET_NAME}/{FILE_PATH}/{FILE_NAME}', file_obj)

    
with open(f'{DESTINATION_FILE}', 'rb') as pickle_file:
    vocab_dict_load = pkl.load(pickle_file)

In [40]:
# len(vocab_dict_load["unique_pids"])

avg_duration_ms_seed_pl = 13000151.68
var_duration_ms_seed_pl = 133092900971233.58
vocab_dict_load['avg_duration_ms_seed_pl']=avg_duration_ms_seed_pl
vocab_dict_load['var_duration_ms_seed_pl']=var_duration_ms_seed_pl

avg_n_songs_pl = 55.21
var_n_songs_pl = 2317.54
vocab_dict_load['avg_n_songs_pl']=avg_n_songs_pl
vocab_dict_load['var_n_songs_pl']=var_n_songs_pl

avg_n_artists_pl = 30.56
var_n_artists_pl = 769.26
vocab_dict_load['avg_n_artists_pl']=avg_n_artists_pl
vocab_dict_load['var_n_artists_pl']=var_n_artists_pl

avg_n_albums_pl = 40.25
var_n_albums_pl = 1305.54
vocab_dict_load['avg_n_albums_pl']=avg_n_albums_pl
vocab_dict_load['var_n_albums_pl']=var_n_albums_pl

avg_artist_pop = 16.08
var_artist_pop = 300.64
vocab_dict_load['avg_artist_pop']=avg_artist_pop
vocab_dict_load['var_artist_pop']=var_artist_pop

avg_duration_ms_songs_pl = 234823.14
var_duration_ms_songs_pl = 5558806228.41
vocab_dict_load['avg_duration_ms_songs_pl']=avg_duration_ms_songs_pl
vocab_dict_load['var_duration_ms_songs_pl']=var_duration_ms_songs_pl

avg_artist_followers = 43337.77
var_artist_followers = 377777790193.57
vocab_dict_load['avg_artist_followers']=avg_artist_followers
vocab_dict_load['var_artist_followers']=var_artist_followers

avg_track_pop = 10.85
var_track_pop = 202.18
vocab_dict_load['avg_track_pop']=avg_track_pop
vocab_dict_load['var_track_pop']=var_track_pop
# vocab_dict_load['unique_pids_string']

In [41]:
vocab_dict_load['unique_pids']

array([     0,      1,      2, ..., 999997, 999998, 999999])

In [42]:
VERSION='test-v1'

In [43]:
BUCKET='spotify-tfrecords-blog'
bucket=client.bucket(BUCKET)
blob = bucket.blob(f'vocabs_stats/vocab_dict_{VERSION}.txt')
pickle_out = pkl.dumps(vocab_dict_load)
blob.upload_from_string(pickle_out)

In [20]:
# bucket=client.bucket(BUCKET)  
# blob = bucket.blob('vocabs/string_vocabs')
# blob.upload_from_filename('string_vocabs')

### Test instances

In [44]:
playlist_test_instancet = {
    'name': np.asarray([b'Best Christmas']),
    'collaborative': np.asarray([b'false']),
    'pid': np.asarray([173671]),
    'description_pl': np.asarray([b'test description']),
    'duration_ms_seed_pl': np.asarray([5458995.]),
    'n_songs_pl': np.asarray([58.]),
    'num_artists_pl': np.asarray([19.]),
    'num_albums_pl': np.asarray([27.]),
    'artist_name_pl': np.asarray([[b'Juan Luis Guerra 4.40', b'Prince Royce', b'Luis Vargas']]),
    'track_uri_pl': np.asarray([[b'spotify:track:1g0IBPZTRP7VYkctJ4Qafg',b'spotify:track:43wUzbYxEFoXugYkgTzMWp']]),
    'track_name_pl': np.asarray([[b'Lover Come Back', b'White Lightning', b'Shake Me Down']]),
    'duration_ms_songs_pl': np.asarray([[245888., 195709., 283906., 271475., 300373., 275173., 236145.,]]),
    'album_name_pl': np.asarray([[b'Silsulim', b'Sara Shara', b'Muzika Vesheket', b'Ba La Lirkod']]),
    'artist_pop_pl': np.asarray([[81., 81., 70., 66., 66., 66., 46., 87.]]),
    'artists_followers_pl': np.asarray([[3.556710e+05, 8.200000e+02, 1.510000e+02, 1.098080e+05,]]),
    'artist_genres_pl': np.asarray([[b"'israeli pop', 'jewish pop'", b"'israeli pop', 'jewish pop'",]]),
    'track_pop_pl': np.asarray([[70, 77, 50, 44, 30, 28, 15, 26, 15, 18, 46, 38,]])
}

In [45]:
candidate_test_instance = {
    'artist_name': np.asarray([b'Hellogoodbye']),
    'track_name': np.asarray([b'When We First Met']),
    'album_name': np.asarray([b'Would It Kill You?']),
    'track_uri': np.asarray([b'Ba La Lirkod']),
    'artist_uri': np.asarray([b'spotify:artist:6GH0NzpthMGxu1mcfAkOde']),
    'album_uri': np.asarray([b'spotify:album:4dHXV7pJs6d8N9ACAMzhIw']),
    'duration_ms': np.asarray([154813.0]),
    'track_pop': np.asarray([45.0]),
    'artist_pop': np.asarray([51.0]),
    'artist_followers':np.asarray([205331.0]),
    'artist_genres': np.asarray([b"'neon pop punk', 'pop punk'"]),
    # 'test': np.asarray([b'test'])
}

# candidate_test_instance = {
#     'artist_name': np.asarray([b'Hellogoodbye']),
#     'track_name': np.asarray([b'When We First Met']),
#     'album_name': np.asarray([b'Would It Kill You?']),
#     'track_uri': np.asarray([b'Ba La Lirkod']),
#     'artist_uri': np.asarray([b'spotify:artist:6GH0NzpthMGxu1mcfAkOde']),
#     'album_uri': np.asarray([b'spotify:album:4dHXV7pJs6d8N9ACAMzhIw']),
#     'duration_ms': np.asarray([154813.0]),
#     'track_pop': np.asarray([45.0]),
#     'artist_pop': np.asarray([51.0]),
#     'artist_followers':np.asarray([205331.0]),
#     'artist_genres': np.asarray([b"'neon pop punk', 'pop punk'"]),
#     # 'test': np.asarray([b'test'])
# }
# pprint(can_test_instance)

# Two-Tower Model

## Playlist (query) Tower

In [46]:
EMBEDDING_DIM = 32
PROJECTION_DIM = 5
SEED = 1234
USE_CROSS_LAYER=True
DROPOUT='False'
DROPOUT_RATE='0.33'

In [47]:
# vocab_dict_load["name"]
len(vocab_dict_load["name"])

74028

In [48]:
vocab_dict_load

{'name': array([b'! 2017 Songs', b'! <3', b'! DJ', ...,
        b'\xf0\x9f\xa6\x84\xf0\x9f\xa6\x84',
        b'\xf0\x9f\xa6\x84\xf0\x9f\xa6\x84\xf0\x9f\xa6\x84',
        b'\xf0\x9f\xa6\x8b\xf0\x9f\xa6\x8b\xf0\x9f\xa6\x8b'], dtype=object),
 'artist_name_can': array([b'!!!', b'!Dela Dap', b'!Distain', ..., b'\xed\x9b\x88 Hun',
        b'\xef\xbc\x92\xef\xbc\x98\xef\xbc\x91\xef\xbc\x94',
        b'\xef\xbc\xad\xef\xbc\xa9\xef\xbc\xb9\xef\xbc\xa1\xef\xbc\xb6\xef\xbc\xa9'],
       dtype=object),
 'track_uri_can': array([b'spotify:track:0000uJA4xCdxThagdLkkLR',
        b'spotify:track:0002yNGLtYSYtc0X6ZnFvp',
        b'spotify:track:00039MgrmLoIzSpuYKurn9', ...,
        b'spotify:track:7zzwsf6tmZETUV26kR7D5z',
        b'spotify:track:7zzxEH0xUl5k3p6IxUfgAO',
        b'spotify:track:7zzyrYnZIfvYAGwl7lRb7X'], dtype=object),
 'artist_uri_can': array([b'spotify:artist:0001ZVMPt41Vwzt1zsmuzp',
        b'spotify:artist:0001cekkfdEBoMlwVQvpLg',
        b'spotify:artist:0001wHqxbF2YYRQxGdbyER', ...,

In [84]:
class Playlist_Model(tf.keras.Model):
    def __init__(self, layer_sizes, vocab_dict):
        super().__init__()

        # ========================================
        # non-sequence playlist features
        # ========================================
        
        # Feature: playlist name
        self.pl_name_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    max_tokens=len(vocab_dict["name"]), # not needed if passing vocab
                    # vocabulary=vocab_dict['name'], 
                    name="pl_name_txt_vectorizer", 
                    ngrams=2
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict["name"]) + 1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="pl_name_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="pl_name_pooling"),
            ], name="pl_name_emb_model"
        )
        
        # Feature: collaborative
        collaborative_vocab = np.array([b'false', b'true'])
        
        self.pl_collaborative_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=collaborative_vocab, 
                    mask_token=None, 
                    name="pl_collaborative_lookup", 
                    output_mode='int'
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(collaborative_vocab) + 1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="pl_collaborative_emb_layer",
                ),
            ], name="pl_collaborative_emb_model"
        )
        
        # Feature: pid
        self.pl_track_uri_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=tf.constant(vocab_dict['track_uri_can']), 
                    mask_token=None, 
                    name="pl_track_uri_lookup", 
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_uri_can'])+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="pl_track_uri_layer",
                ),
            ], name="pl_track_uri_emb_model"
        )
        
        # Feature: description_pl
#         self.pl_description_text_embedding = tf.keras.Sequential(
#             [
#                 tf.keras.layers.TextVectorization(
#                     max_tokens=len(vocab_dict["description_pl"]), # not needed if passing vocab
#                     # vocabulary=tf.constant(vocab_dict['description_pl']), 
#                     name="description_pl_vectorizer", 
#                 ),
#                 tf.keras.layers.Embedding(
#                     input_dim=len(vocab_dict["description_pl"]) + 1,
#                     output_dim=EMBEDDING_DIM,
#                     mask_zero=False,
#                     name="description_pl_emb_layer",
#                 ),
#                 tf.keras.layers.GlobalAveragePooling1D(name="description_pl_pooling"),
#             ], name="pl_description_emb_model"
#         )
        
        # Feature: duration_ms_seed_pl                      
        # TODO: Noramlize or Descritize?
        # duration_ms_seed_pl_buckets = np.linspace(
        #     vocab_dict['min_duration_ms_seed_pl'], 
        #     vocab_dict['max_duration_ms_seed_pl'], 
        #     num=1000
        # )
        # self.duration_ms_seed_pl_embedding = tf.keras.Sequential(
        #     [
        #         tf.keras.layers.Discretization(duration_ms_seed_pl_buckets.tolist()),
        #         tf.keras.layers.Embedding(
        #             input_dim=len(duration_ms_seed_pl_buckets) + 1, 
        #             output_dim=EMBEDDING_DIM, 
        #             name="duration_ms_seed_pl_emb_layer",
        #         )
        #     ], name="duration_ms_seed_pl_emb_model"
        # )
        # self.duration_ms_seed_pl_normalization = tf.keras.layers.Normalization(
        #     mean=vocab_dict['avg_duration_ms_seed_pl'],
        #     variance=vocab_dict['var_duration_ms_seed_pl'],
        #     axis=None
        # )
        
        # Feature: n_songs_pl
        # TODO: Noramlize or Descritize?
        n_songs_pl_buckets = np.linspace(
            vocab_dict['min_n_songs_pl'], 
            vocab_dict['max_n_songs_pl'], 
            num=100
        )
        self.n_songs_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Discretization(n_songs_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(n_songs_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM, 
                    name="n_songs_pl_emb_layer",
                )
            ], name="n_songs_pl_emb_model"
        )
        # self.n_songs_pl_normalization = tf.keras.layers.Normalization(
        #     mean=vocab_dict['avg_n_songs_pl'],
        #     variance=vocab_dict['var_n_songs_pl'],
        #     axis=None
        # )
        
        # Feature: num_artists_pl
        # TODO: Noramlize or Descritize?
        n_artists_pl_buckets = np.linspace(
            vocab_dict['min_n_artists_pl'], 
            vocab_dict['max_n_artists_pl'], 
            num=100
        )
        self.n_artists_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Discretization(n_artists_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(n_artists_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM, 
                    name="n_artists_pl_emb_layer",
                    mask_zero=False
                )
            ], name="n_artists_pl_emb_model"
        )
        # self.n_artists_pl_normalization = tf.keras.layers.Normalization(
        #     mean=vocab_dict['avg_n_artists_pl'],
        #     variance=vocab_dict['var_n_artists_pl'],
        #     axis=None
        # )
        
        # Feature: num_albums_pl
        n_albums_pl_buckets = np.linspace(
            vocab_dict['min_n_albums_pl'], 
            vocab_dict['max_n_albums_pl'],
            num=100
        )
        self.n_albums_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Discretization(n_albums_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(n_albums_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM, 
                    name="n_albums_pl_emb_layer",
                )
            ], name="n_albums_pl_emb_model"
        )
        # self.n_albums_pl_normalization = tf.keras.layers.Normalization(
        #     mean=vocab_dict['avg_n_albums_pl'],
        #     variance=vocab_dict['var_n_albums_pl'],
        #     axis=None
        # )
        
        # ========================================
        # sequence playlist features
        # ========================================
        
        # Feature: artist_name_pl
        self.artist_name_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['artist_name_pl'], mask_token=''),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['artist_name_pl']) + 2, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_name_pl_emb_layer",
                    mask_zero=False
                ),
                # tf.keras.layers.GlobalAveragePooling1D(EMBEDDING_DIM, name="artist_name_conv1d"), # GlobalAveragePooling1D
                tf.keras.layers.GRU(EMBEDDING_DIM, name="artist_name_gru"),
            ], name="artist_name_pl_emb_model"
        )
        
        # Feature: track_uri_pl
        # 2.2M unique
        self.track_uri_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['track_uri_can'], mask_token=''),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_uri_can']) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="track_uri_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_uri_1d"),
            ], name="track_uri_pl_emb_model"
        )
        
        # Feature: track_name_pl
        self.track_name_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['track_name_pl'], 
                    name="track_name_pl_lookup",
                    output_mode='int',
                    mask_token=''
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_name_pl']) + 2, 
                    output_dim=EMBEDDING_DIM,
                    name="track_name_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_name_pl_1d"),
            ], name="track_name_pl_emb_model"
        )
        
        Feature: duration_ms_songs_pl
        duration_ms_songs_pl_buckets = np.linspace(
            vocab_dict['min_duration_ms_songs_pl'], 
            vocab_dict['max_duration_ms_songs_pl'], 
            num=100
        )
        self.duration_ms_songs_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.Discretization(duration_ms_songs_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(duration_ms_songs_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="duration_ms_songs_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="duration_ms_songs_pl_emb_layer_pl_1d"),
            ], name="duration_ms_songs_pl_emb_model"
        )
        
        # Feature: album_name_pl
        self.album_name_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=tf.constant(vocab_dict['album_name_pl']), 
                    mask_token=None, 
                    name="album_name_pl_lookup"
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['album_name_pl']) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="album_name_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="album_name_pl_emb_layer_1d"),
            ], name="album_name_pl_emb_model"
        )
        
        # Feature: artist_pop_pl
        artist_pop_pl_buckets = np.linspace(
            vocab_dict['min_artist_pop'], 
            vocab_dict['max_artist_pop'], 
            num=10
        )
        self.artist_pop_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.Discretization(artist_pop_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(artist_pop_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_pop_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_pop_1d"),
            ], name="artist_pop_pl_emb_model"
        )
        
        # Feature: artists_followers_pl
        artists_followers_pl_buckets = np.linspace(
            vocab_dict['min_artist_followers'], 
            vocab_dict['max_artist_followers'], 
            num=10
        )
        self.artists_followers_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.Discretization(artists_followers_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(artists_followers_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="artists_followers_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artists_followers_pl_1d"),
            ], name="artists_followers_pl_emb_model"
        )
        
        # Feature: track_pop_pl
        track_pop_pl_buckets = np.linspace(
            vocab_dict['min_track_pop'], 
            vocab_dict['max_track_pop'], 
            num=10
        )
        self.track_pop_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(dtype=tf.float32),
                tf.keras.layers.Discretization(track_pop_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(track_pop_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="track_pop_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_pop_pl_1d"),
            ], name="track_pop_pl_emb_model"
        )
        
        # Feature: artist_genres_pl
        self.artist_genres_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['artist_genres_pl'], mask_token=''),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['artist_genres_pl']) + 2, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_genres_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_genres_pl_1d"),
            ], name="artist_genres_pl_emb_model"
        )
        
        # ========================================
        # dense and cross layers
        # ========================================

        # Cross Layers
        if USE_CROSS_LAYER:
            self._cross_layer = tfrs.layers.dcn.Cross(
                projection_dim=PROJECTION_DIM,
                kernel_initializer="glorot_uniform", 
                name="pl_cross_layer"
            )
        else:
            self._cross_layer = None
            
        # Dense Layers
        self.dense_layers = tf.keras.Sequential(name="pl_dense_layers")
        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    activation="relu", 
                    kernel_initializer=initializer,
                )
            )
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(DROPOUT_RATE))
                
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    kernel_initializer=initializer
                )
            )
        ### ADDING L2 NORM AT THE END
        self.dense_layers.add(
            tf.keras.layers.Lambda(
                lambda x: tf.nn.l2_normalize(
                    x, 1, epsilon=1e-12, name="normalize_dense"
                )
            )
        )
        
    # ========================================
    # call
    # ========================================
    def call(self, data):
        '''
        The call method defines what happens when
        the model is called
        '''
        
        all_embs = tf.concat(
            [
                self.pl_name_text_embedding(data['name']),
                self.pl_collaborative_embedding(data['collaborative']),
                self.pl_track_uri_embedding(data["track_uri_can"]),
                # self.pl_description_text_embedding(data['description_pl']),
                # self.duration_ms_seed_pl_embedding(data["duration_ms_seed_pl"]),
                # tf.reshape(self.duration_ms_seed_pl_normalization(data["duration_ms_seed_pl"]), (-1, 1))      # Normalize or Discretize?
                self.n_songs_pl_embedding(data["n_songs_pl"]),
                # tf.reshape(self.n_songs_pl_normalization(data["n_songs_pl"]), (-1, 1))                        # Normalize or Discretize?
                self.n_artists_pl_embedding(data['num_artists_pl']),
                # tf.reshape(self.n_artists_pl_normalization(data["num_artists_pl"]), (-1, 1))                  # Normalize or Discretize?
                self.n_albums_pl_embedding(data["num_albums_pl"]),
                # tf.reshape(self.n_albums_pl_normalization(data["num_albums_pl"]), (-1, 1))                    # Normalize or Discretize?
                
                # sequence features
                # data["pos_pl"],
                self.artist_name_pl_embedding(data["artist_name_pl"]),
                self.track_uri_pl_embedding(data["track_uri_pl"]),
                self.track_name_pl_embedding(data["track_name_pl"]),
                self.duration_ms_songs_pl_embedding(data["duration_ms_songs_pl"]),
                self.album_name_pl_embedding(data["album_name_pl"]),
                self.artist_pop_pl_embedding(data["artist_pop_pl"]),
                self.artists_followers_pl_embedding(data["artists_followers_pl"]),
                self.track_pop_pl_embedding(data["track_pop_pl"]),
                self.artist_genres_pl_embedding(data["artist_genres_pl"]),
            ], axis=1)
        
        # Build Cross Network
        if self._cross_layer is not None:
            cross_embs = self._cross_layer(all_embs)
            return self.dense_layers(cross_embs)
        else:
            return self.dense_layers(all_embs)

### test playlist tower

In [85]:
layer_sizes=[64,32]

test_playlist_model = Playlist_Model(layer_sizes,vocab_dict_load)

test_playlist_model.pl_name_text_embedding.layers[0].adapt(parsed_dataset_padded.map(lambda x: x['name']).batch(1000))

print("Adapts complete for name")

# test_playlist_model.pl_description_text_embedding.layers[0].adapt(parsed_dataset_padded.map(lambda x: x['description_pl']).batch(1000))

# print("Adapts complete for description")

Adapts complete for name


In [512]:
test_playlist_model.pl_name_text_embedding.layers[0].get_vocabulary()

['',
 '[UNK]',
 'music',
 'country',
 'rock',
 'chill',
 'summer',
 'songs',
 'party',
 'good',
 'jams',
 'rap',
 'playlist',
 'the',
 'new',
 'my',
 'old',
 'christmas',
 'oldies',
 'mix',
 'workout',
 '2017',
 'throwback',
 '2016',
 'vibes',
 'classic',
 'car',
 'work',
 'school',
 'road',
 '90s',
 '2015',
 'worship',
 'lit',
 'dance',
 'love',
 'up',
 'stuff',
 'best',
 'feels',
 'pop',
 'wedding',
 'hop',
 'trip',
 'throwbacks',
 'hip',
 'out',
 '80s',
 'it',
 'edm',
 'of',
 'disney',
 'classics',
 'time',
 'hip hop',
 'road trip',
 'old school',
 'classic rock',
 'gym',
 'fall',
 'happy',
 'tunes',
 'alternative',
 'spring',
 'christian',
 'slow',
 'jamz',
 '17',
 'random',
 'all',
 'for',
 'run',
 'running',
 'feel',
 '2014',
 'rb',
 'study',
 'indie',
 'i',
 'me',
 'spanish',
 'feel good',
 '2',
 'house',
 'sad',
 'hype',
 'sleep',
 'driving',
 'day',
 'chillin',
 'beach',
 'you',
 'mood',
 'back',
 '16',
 'jazz',
 'shower',
 'but',
 'favorites',
 'mellow',
 'hits',
 'reggae',
 

In [ ]:
vocab_dict_load['name_voacb'] = test_playlist_model.pl_name_text_embedding.layers[0].get_vocabulary()

### Note to JT - Description was giving me an issue. It is mostly null and throwing NANs - I just commented it out for now in case you wanted to get it to work. Otherwise we are training on the small data just fine!!

In [86]:
#test with the source batched datset
batched_dataset = parsed_dataset_padded.batch(2) # @JT this seems to matter on the data - I think there are bad lookups maybe
for x in batched_dataset.take(1):
    print(test_playlist_model(x))

tf.Tensor(
[[ 0.39000463 -0.10863615  0.23910482  0.03924488  0.1732322   0.0749235
   0.17243938 -0.09064244 -0.17611617  0.14259705 -0.00373934 -0.10498305
  -0.21623299  0.03421612  0.08665393  0.07933064  0.09225872  0.08139911
   0.37539792 -0.03706365 -0.2434901   0.10924795 -0.28263348 -0.33317617
   0.08152765  0.07537527 -0.04569072  0.33724004 -0.10783918 -0.07342485
   0.08463598 -0.08841529]
 [ 0.617287   -0.10422025 -0.04544194 -0.05393054  0.17941467 -0.02629733
   0.12981717  0.09263929 -0.20038012  0.01568789  0.07754222 -0.03748523
  -0.17163222  0.16355038  0.07196476  0.1885697   0.07738508 -0.00621079
   0.13480452  0.1433457  -0.08734424  0.06575251  0.13355383 -0.17948589
   0.2244298  -0.26892054  0.14537346  0.1857176  -0.22845145 -0.04463708
   0.06584466 -0.2376232 ]], shape=(2, 32), dtype=float32)


In [87]:
#diagnosing NANs
batched_dataset = parsed_dataset_padded.skip(8).batch(1) # @JT this seems to matter on the data - I think there are bad lookups maybe
for x in batched_dataset.take(1):
    print(x['description_pl'])

tf.Tensor([b''], shape=(1,), dtype=string)


In [88]:
test_playlist_model.summary(expand_nested=True)

Model: "playlist__model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pl_name_emb_model (Sequenti  (None, 32)               2368928   
 al)                                                             
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| pl_name_txt_vectorizer (Tex  (None, None)           0         |
| tVectorization)                                               |
|                                                               |
| pl_name_emb_layer (Embeddin  (None, None, 32)       2368928   |
| g)                                                            |
|                                                               |
| pl_name_pooling (GlobalAver  (None, 32)             0         |
| agePooling1D)                                                 |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
 pl_collaborative_emb_model   (2, 32)           

## Track (candidate) Tower 

In [89]:
EMBEDDING_DIM = 32
PROJECTION_DIM = 5
SEED = 1234
USE_CROSS_LAYER=True
DROPOUT='False'
DROPOUT_RATE='0.33'

# candidate_test_instance

In [99]:
class Candidate_Track_Model(tf.keras.Model):
    def __init__(self, layer_sizes, vocab_dict):
        super().__init__()
        
        # ========================================
        # Candidate features
        # ========================================
        
        # Feature: artist_name_can
        self.artist_name_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    vocabulary=vocab_dict["artist_name_can"],
                    name="artist_name_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict["artist_name_can"])+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="artist_name_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_name_can_pooling"),
            ], name="artist_name_can_emb_model"
        )
        
        # Feature: track_name_can
        self.track_name_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    vocabulary=vocab_dict["track_name_can"],
                    name="track_name_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict["track_name_can"])+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="track_name_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_name_can_pooling"),
            ], name="track_name_can_emb_model"
        )
        
        # Feature: album_name_can
        self.album_name_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    vocabulary=vocab_dict["album_name_can"],
                    name="album_name_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict["album_name_can"])+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="album_name_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="album_name_can_pooling"),
            ], name="album_name_can_emb_model"
        )
        
        # Feature: artist_uri_can
        self.artist_uri_can_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Hashing(num_bins=200_000),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['artist_uri_can'])+1, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_uri_can_emb_layer",
                ),
            ], name="artist_uri_can_emb_model"
        )
        
        # Feature: track_uri_can
        self.track_uri_can_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Hashing(num_bins=200_000),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_uri_can'])+1, 
                    output_dim=EMBEDDING_DIM,
                    name="track_uri_can_emb_layer",
                ),
            ], name="track_uri_can_emb_model"
        )
        
        # Feature: album_uri_can
        self.album_uri_can_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Hashing(num_bins=200_000),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['album_uri_can'])+1, 
                    output_dim=EMBEDDING_DIM,
                    name="album_uri_can_emb_layer",
                ),
            ], name="album_uri_can_emb_model"
        )
        
        # Feature: duration_ms_can
        self.duration_ms_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_duration_ms_songs_pl'],
            variance=vocab_dict['var_duration_ms_songs_pl'],
            axis=None
        )
        
        # Feature: track_pop_can
        self.track_pop_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_track_pop'],
            variance=vocab_dict['var_track_pop'],
            axis=None
        )
        
        # Feature: artist_pop_can
        self.artist_pop_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_artist_pop'],
            variance=vocab_dict['var_artist_pop'],
            axis=None
        )
        
        # Feature: artist_followers_can
        self.artist_followers_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_artist_followers'],
            variance=vocab_dict['var_artist_followers'],
            axis=None
        )
        
        # Feature: artist_genres_can
        self.artist_genres_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    vocabulary=vocab_dict["artist_genres_can"],
                    name="artist_genres_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict["artist_genres_can"])+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="artist_genres_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_genres_can_pooling"),
            ], name="artist_genres_can_emb_model"
        )
        
        # ========================================
        # Dense & Cross Layers
        # ========================================
        
        # Cross Layers
        if USE_CROSS_LAYER:
            self._cross_layer = tfrs.layers.dcn.Cross(
                projection_dim=PROJECTION_DIM,
                kernel_initializer="glorot_uniform", 
                name="can_cross_layer"
            )
        else:
            self._cross_layer = None
        
        # Dense Layer
        self.dense_layers = tf.keras.Sequential(name="candidate_dense_layers")
        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    activation="relu", 
                    kernel_initializer=initializer,
                )
            )
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(DROPOUT_RATE))
                
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    kernel_initializer=initializer
                )
            )
            
    # ========================================
    # Call Function
    # ========================================
            
    def call(self, data):
        
        all_embs = tf.concat(
            [
                self.artist_name_can_text_embedding(data['artist_name_can']),  #TODO: removed `_can` from feature name
                self.track_name_can_text_embedding(data['track_name_can']),  #TODO: removed `_can` from feature name
                self.album_name_can_text_embedding(data['album_name_can']),  #TODO: removed `_can` from feature name
                self.artist_uri_can_embedding(data['artist_uri_can']),  #TODO: removed `_can` from feature name
                self.track_uri_can_embedding(data['track_uri_can']),  #TODO: removed `_can` from feature name
                self.album_uri_can_embedding(data['album_uri_can']),  #TODO: removed `_can` from feature name
                tf.reshape(self.duration_ms_can_normalized(data["duration_ms_can"]), (-1, 1)),  #TODO: removed `_can` from feature name
                tf.reshape(self.track_pop_can_normalized(data["track_pop_can"]), (-1, 1)),  #TODO: removed `_can` from feature name
                tf.reshape(self.artist_pop_can_normalized(data["artist_pop_can"]), (-1, 1)),  #TODO: removed `_can` from feature name
                tf.reshape(self.artist_followers_can_normalized(data["artist_followers_can"]), (-1, 1)),  #TODO: removed `_can` from feature name
                self.artist_genres_can_text_embedding(data['album_uri_can']),  #TODO: removed `_can` from feature name
            ], axis=1
        )
        
        
        
        # return self.dense_layers(all_embs)
                # Build Cross Network
        if self._cross_layer is not None:
            cross_embs = self._cross_layer(all_embs)
            return self.dense_layers(cross_embs)
        else:
            return self.dense_layers(all_embs)

In [100]:
layer_sizes=[64,32]

test_can_track_model = Candidate_Track_Model(layer_sizes, vocab_dict_load)

# can_result = test_can_track_model([candidate_test_instance])

# print(f"Shape of can_result: {can_result.shape}")
# can_result

In [103]:
for x in candidate_dataset.take(1):
    print(x)

tf.Tensor(b"\n\xc8\x04\n1\n\x0ealbum_name_can\x12\x1f\n\x1d\n\x1bThe Sound of Everything Rmx\n9\n\ralbum_uri_can\x12(\n&\n$spotify:album:4a8tMD6qq6GUuUwNae38VI\n \n\x14artist_followers_can\x12\x08\x12\x06\n\x04 \x92\x87H\n}\n\x11artist_genres_can\x12h\nf\nd'downtempo', 'electronica', 'funk', 'latin alternative', 'nu jazz', 'nu-cumbia', 'trip hop', 'world'\n\x1e\n\x0fartist_name_can\x12\x0b\n\t\n\x07Quantic\n\x1a\n\x0eartist_pop_can\x12\x08\x12\x06\n\x04\x00\x00\x80B\n;\n\x0eartist_uri_can\x12)\n'\n%spotify:artist:5ZMwoAjeDtLJ0XRwRTgaK8\n\x1b\n\x0fduration_ms_can\x12\x08\x12\x06\n\x04@o\x82H\nK\n\x0etrack_name_can\x129\n7\n5The Sound of Everything - Watch TV & Se\xc3\xb1orlobo Remix\n\x19\n\rtrack_pop_can\x12\x08\x12\x06\n\x04\x00\x00TB\n9\n\rtrack_uri_can\x12(\n&\n$spotify:track:27CDzo2P7Mf3dKoa76tNxb", shape=(), dtype=string)


In [104]:
#test with the source batched datset and candidate dataset
batched_dataset = parsed_candidate_dataset.batch(2)
for x in batched_dataset.take(1):
    print(test_can_track_model(x))

tf.Tensor(
[[ 0.0007672   0.03160781  0.2735462  -0.31936097  0.12938203 -0.02584392
   0.11680165  0.20058246 -0.28498572 -0.28866202 -0.45320028  0.21031171
   0.5178399  -0.5153135  -0.31266952 -0.13795605 -0.12313841 -0.03808884
   0.02938222  0.19159947 -0.39058346 -0.01792859  0.1305196  -0.4349187
   0.33412743  0.11348049 -0.11954457  0.34586486  0.10583854  0.50431657
   0.02920749  0.41339418]
 [-0.35195065  0.09021632  0.2524035  -0.13075542  0.32896525 -0.1617768
  -0.15252593  0.28818858 -0.3332807   0.17067063 -0.3194032   0.14723924
   0.3079238  -0.21826419 -0.37196046 -0.01728063 -0.16132183 -0.48820788
  -0.21429391 -0.00109622 -0.42549205  0.13567866  0.33385926 -0.38768464
   0.17946196  0.16186784  0.01078631  0.16730186  0.05512255  0.44132572
   0.08475317  0.22834884]], shape=(2, 32), dtype=float32)


In [501]:
test_can_track_model.summary(expand_nested=True)

Model: "candidate__track__model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 artist_name_can_emb_model (  (None, 32)               9206752   
 Sequential)                                                     
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| artist_name_can_txt_vectori  (None, None)           0         |
| zer (TextVectorization)                                       |
|                                                               |
| artist_name_can_emb_layer (  (None, None, 32)       9206752   |
| Embedding)                                                    |
|                                                               |
| artist_name_can_pooling (Gl  (None, 32)             0         |
| obalAveragePooling1D)                                         |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
 track_name_can_emb_model (S  (None, 32)

## Combined Model

In [502]:
# pl_can_concat = tf.concat([pl_result,can_result], axis=1)

# print(f"Shape of pl_can_concat: {pl_can_concat.shape[1]}")
# pl_can_concat

In [105]:
# %%writefile -a vertex_train/trainer/task.py

class TheTwoTowers(tfrs.models.Model):

    def __init__(self, layer_sizes, vocab_dict_load):
        super().__init__()
        
        self.query_tower = Playlist_Model(layer_sizes, vocab_dict_load)
        
        self.candidate_tower = Candidate_Track_Model(layer_sizes, vocab_dict_load)
        
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=parsed_candidate_dataset.batch(128).cache().map(self.candidate_tower)
            )
        )
        
    def compute_loss(self, data, training=False):
        query_embeddings = self.query_tower(data)
        candidate_embeddings = self.candidate_tower(data)

        return self.task(
            query_embeddings, 
            candidate_embeddings, 
            compute_metrics=not training
        ) # turn off metrics to save time on training

In [106]:
layer_sizes=[64,32]

model = TheTwoTowers(layer_sizes, vocab_dict_load)

model.compile(optimizer=tf.keras.optimizers.Adagrad(.1))

In [107]:
#do the adapt
model.query_tower.pl_name_text_embedding.layers[0].adapt(parsed_dataset_padded.map(lambda x: x['name']).batch(1000))


In [108]:
## Quick look at the layers
print("Playlist (query) Tower:")

for i, l in enumerate(model.query_tower.layers):
    print(i, l.name)

Playlist (query) Tower:
0 pl_name_emb_model
1 pl_collaborative_emb_model
2 pl_track_uri_emb_model
3 n_songs_pl_emb_model
4 n_artists_pl_emb_model
5 n_albums_pl_emb_model
6 artist_name_pl_emb_model
7 track_uri_pl_emb_model
8 track_name_pl_emb_model
9 duration_ms_songs_pl_emb_model
10 album_name_pl_emb_model
11 artist_pop_pl_emb_model
12 artists_followers_pl_emb_model
13 track_pop_pl_emb_model
14 artist_genres_pl_emb_model
15 pl_cross_layer
16 pl_dense_layers


In [109]:
print("Track (candidate) Tower:")
for i, l in enumerate(model.candidate_tower.layers):
    print(i, l.name)

Track (candidate) Tower:
0 artist_name_can_emb_model
1 track_name_can_emb_model
2 album_name_can_emb_model
3 artist_uri_can_emb_model
4 track_uri_can_emb_model
5 album_uri_can_emb_model
6 normalization_12
7 normalization_13
8 normalization_14
9 normalization_15
10 artist_genres_can_emb_model
11 can_cross_layer
12 candidate_dense_layers


# Local Training

In [110]:
tf.random.set_seed(42)
shuffled_parsed_ds = parsed_dataset_padded.shuffle(10_000, seed=42, reshuffle_each_iteration=False)

TRAIN_small = shuffled_parsed_ds.take(80_000).batch(128)
TEST_small = shuffled_parsed_ds.skip(80_000).take(20_000).batch(128)

In [514]:
TEST_small

<BatchDataset element_spec={'album_name_can': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'album_name_seed_track': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'album_uri_seed_track': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'artist_followers_seed_track': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'artist_genres_seed_track': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'artist_name_seed_track': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'artist_pop_seed_track': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(sha

In [111]:
import time    

start_time = time.time()

NUM_EPOCHS = 1
layer_history = model.fit(
    TRAIN_small,
    validation_data=TEST_small,
    # validation_freq=5,
    epochs=NUM_EPOCHS,
    # callbacks=tensorboard_cb,
    # verbose=1
)

train_time = time.time() - start_time

print(f"Training for {NUM_EPOCHS} epoch, ran for: {train_time:.0} seconds")
accuracy = layer_history.history["factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top 100 categorical accuracy: {accuracy}")

24/24 [==============================] - 19s 361ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 11876.6699 - regularization_loss: 0.0000e+00 - total_loss: 11876.6699
Training for 1 epoch, ran for: 2e+01 seconds
Top 100 categorical accuracy: 0.0


# Local Testing

## Loading SavedModels

In [526]:
candidate_tower_uri = 'gs://spotify-tfrs-dir/v2/run-20220920-210334/candidate_tower'
loaded_candidate_model = tf.saved_model.load(candidate_tower_uri)

### Candidate Model

In [527]:
print(list(loaded_candidate_model.signatures.keys()))

['serving_default']


In [528]:
infer = loaded_candidate_model.signatures["serving_default"]
print(infer.structured_outputs)

{'output_1': TensorSpec(shape=(None, 32), dtype=tf.float32, name='output_1')}


In [530]:
loaded_candidate_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, album_name_can, album_uri_can, artist_followers_can, artist_genres_can, artist_name_can, artist_pop_can, artist_uri_can, duration_ms_can, track_name_can, track_pop_can, track_uri_can) at 0x7F1A85BA2C10>})


In [531]:
predict2 = loaded_candidate_model.signatures['serving_default']
predict2.output_shapes

{'output_1': TensorShape([None, 32])}

In [533]:
parsed_candidate_dataset

<MapDataset element_spec={'album_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'duration_ms_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'track_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'track_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'track_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [534]:
embs_iter = parsed_candidate_dataset.batch(1).map(
    lambda data: predict2(
        artist_name_can = data["artist_name_can"],
        track_name_can = data['track_name_can'],
        album_name_can = data['album_name_can'],
        track_uri_can = data['track_uri_can'],
        artist_uri_can = data['artist_uri_can'],
        album_uri_can = data['album_uri_can'],
        duration_ms_can = data['duration_ms_can'],
        track_pop_can = data['track_pop_can'],
        artist_pop_can = data['artist_pop_can'],
        artist_followers_can = data['artist_followers_can'],
        artist_genres_can = data['artist_genres_can']
    )
)

embs = []
for emb in embs_iter:
    embs.append(emb)

In [540]:
print(f"Length of embs: {len(embs)}")
embs[0]

Length of embs: 166827


{'output_1': <tf.Tensor: shape=(1, 32), dtype=float32, numpy=
 array([[ 0.32745814,  0.15835902, -0.62232316, -0.18282643,  0.05425396,
          0.08093273, -0.36878368, -0.6777717 , -0.28926852, -0.44578883,
          0.03769037,  0.2013096 ,  0.01650199, -0.38547963, -0.2734376 ,
          0.16123655,  0.12652676, -0.09455037,  0.3709236 , -0.04336764,
         -0.20310198,  0.21418957, -0.44912496, -0.16798183, -0.21492694,
          0.04033846,  0.14083184,  0.25597924,  0.20490994,  0.18837534,
         -0.15261272,  0.06471566]], dtype=float32)>}

In [541]:
cleaned_embs = [x['output_1'].numpy()[0] for x in embs] #clean up the output

print(f"Length of cleaned_embs: {len(cleaned_embs)}")
cleaned_embs[0]

Length of cleaned_embs: 166827


array([ 0.32745814,  0.15835902, -0.62232316, -0.18282643,  0.05425396,
        0.08093273, -0.36878368, -0.6777717 , -0.28926852, -0.44578883,
        0.03769037,  0.2013096 ,  0.01650199, -0.38547963, -0.2734376 ,
        0.16123655,  0.12652676, -0.09455037,  0.3709236 , -0.04336764,
       -0.20310198,  0.21418957, -0.44912496, -0.16798183, -0.21492694,
        0.04033846,  0.14083184,  0.25597924,  0.20490994,  0.18837534,
       -0.15261272,  0.06471566], dtype=float32)

In [553]:
# clean product IDs
track_uris = [x['track_uri_can'].numpy() for x in parsed_candidate_dataset]
track_uris[0]

b'spotify:track:6KhJeYLg1AimCQjH6ii1Al'

In [554]:
track_uris[0]

b'spotify:track:6KhJeYLg1AimCQjH6ii1Al'

In [557]:
track_uris_cleaned = [str(z).replace("b'","").replace("'","") for z in track_uris]
track_uris_cleaned[0]

'spotify:track:6KhJeYLg1AimCQjH6ii1Al'

In [558]:
print(f"Length of track_uris: {len(track_uris)}")
print(f"Length of track_uris_cleaned: {len(track_uris_cleaned)}")

Length of track_uris: 166827
Length of track_uris_cleaned: 166827


### Write Index Json File

In [559]:
VERSION = 'local-v1'
TIMESTAMP = '092022'

embeddings_index_filename = f'candidate_embeddings_{VERSION}_{TIMESTAMP}.json'

with open(f'{embeddings_index_filename}', 'w') as f:
    for prod, emb in zip(track_uris_cleaned, cleaned_embs):
        f.write('{"id":"' + str(prod) + '",')
        f.write('"embedding":[' + ",".join(str(x) for x in list(emb)) + "]}")
        f.write("\n")

In [564]:
# import json

# with open('candidate_embeddings_local-v1_092022.json', 'r') as f:
#     data = json.load(f)

### Query Model

In [565]:
query_tower_uri = 'gs://spotify-tfrs-dir/v2/run-20220920-210334/query_tower'
loaded_query_model = tf.saved_model.load(query_tower_uri)

In [566]:
loaded_query_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, album_name_can, album_name_pl, album_name_seed_track, album_uri_can, album_uri_seed_track, artist_followers_can, artist_followers_seed_track, artist_genres_can, artist_genres_pl, artist_genres_seed_track, artist_name_can, artist_name_pl, artist_name_seed_track, artist_pop_can, artist_pop_pl, artist_pop_seed_track, artist_uri_can, artist_uri_seed_track, artists_followers_pl, collaborative, description_pl, duration_ms_can, duration_ms_seed_pl, duration_ms_songs_pl, duration_seed_track, n_songs_pl, name, num_albums_pl, num_artists_pl, pid, pos_seed_track, track_name_can, track_name_pl, track_name_seed_track, track_pop_can, track_pop_pl, track_pop_seed_track, track_uri_can, track_uri_pl, track_uri_seed_track) at 0x7F1935582D50>})